In [43]:
# #### Pre Installs
# import os
# os.environ['openAIAPIKey'] = ''
# os.environ['serpAPIKey'] = ''
!pip install google-search-results

In [74]:
appConfig = {
    "NoOfLinks" : 3,
    "oaSystemprompt" : "You are an assistant that analyzes the contents of a website and extracts requested details in a json format with provided keys and ignoring and other text. Respond in markdown.",
    "oaUserprompt" : ""
}

In [34]:
import os

class getApiKeys:

  def __init__(self):
    pass

  def getApiKeys(self,apiKey):
    return os.environ.get(apiKey)

In [64]:
import serpapi
import json
from serpapi import GoogleSearch # Add this import statement

class searchForLinks:
  def __init__(self):
    pass

  def getSearchResults(self,searchWord):
    searpApiKey = getApiKeys()
    params = {
      "q": searchWord,
      "hl": "en",
      "gl": "us",
      "google_domain": "google.com",
      "api_key": ''
    }

    search = GoogleSearch(params)
    results = search.get_dict()
    linksList = []

    if 'organic_results' in results:
        for res in results['organic_results']:
            if 'link' in res:  # Check if 'link' key exists in each result dictionary
                linksList.append(res['link'])
    else:
        print("No organic results found in the response.")

    return linksList[:appConfig['NoOfLinks']]

In [69]:
sfl = searchForLinks()
sfl.getSearchResults('american express credit card terms')

['https://www.americanexpress.com/us/credit-cards/features-benefits/additional-card/terms-and-conditions.html',
 'https://www.americanexpress.com/content/dam/amex/in/legal/mitc/Card_Member_Aggreement.pdf',
 'https://www.americanexpress.com/en-us/benefits/offers/partner-terms/']

In [62]:
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display


class webCrawler:

  def __init__(self):
    pass

  def crawlresults(self,srch):
    sfl = searchForLinks()
    linktocrawl = sfl.getSearchResults(srch)

    crawlJson = {}
    for links in linktocrawl:
      crawlJson.update(self.crawl(links))

    return crawlJson

  def crawl(self, url: str):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    title = soup.title.string if soup.title else "Title not found"

    if soup.body:
      for irrelevant in soup.body(["script", "style","img","input"]):
        irrelevant.decompose()
      text = soup.body.get_text(separator="\n", strip=True)
    else:
      text = ""

    return {title : text}

    # links = [link.get('href') for link in soup.find_all('a')]
    # self.links = [link for link in links if link]

In [76]:
import os
import requests
# from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI
import time


class summarizer:

  def __init__(self):
    pass

  def user_prompt_for(self,title,text):
      user_prompt = f"You are looking at a website titled {title}"
      user_prompt += "The contents of this website is as follows; \
                      please find any value about APY, Terms rewards and respond as as JSON with following format {'apy':'','fees':''}. \
                      ignore anything else.\n\n"
      user_prompt += text
      return user_prompt

  def messages_for(self,content):
      return [
          {"role": "system", "content": appConfig['oaSystemprompt']},
          {"role": "user", "content": content}
      ]

  def summarize(self,url):
      searpApiKey = getApiKeys()
      api_key = searpApiKey.getApiKeys('openAIAPIKey')

      openai = OpenAI(api_key=api_key)

      crawl = webCrawler()
      contents = crawl.crawlresults(url)

      responses = []

      for content in contents:
        user_prompt = self.user_prompt_for(content,contents[content])
        messages = self.messages_for(content)

        response = openai.chat.completions.create(
            model = "gpt-4o-mini",
            messages = messages
        )
        responses.append(response.choices[0].message.content)

      return responses


In [77]:
extract = summarizer()
results = extract.summarize('chase freedom credit card')
for I in results:
  display(Markdown(I))

```json
{
  "title": "Chase Freedom",
  "category": "Credit Cards",
  "website": "Chase.com"
}
```

```json
{
  "card_name": "Chase Freedom Unlimited",
  "issuer": "Chase",
  "rewards": {
    "cash_back": "1.5% on all purchases",
    "bonus_offer": "Earn a $200 bonus after you spend $500 on purchases in the first 3 months from account opening",
    "categories": {
      "dining": "3% on dining at restaurants, including takeout and eligible delivery services",
      "drugs": "3% at drugstores",
      "travel": "5% on travel purchased through Chase Ultimate Rewards"
    }
  },
  "annual_fee": "$0",
  "foreign_transaction_fee": "$0",
  "intro_offer": "0% intro APR on purchases for 15 months, then a variable APR of 19.24% - 27.49%",
  "credit_score_needed": "Good to Excellent",
  "other_benefits": [
    "Purchase protection",
    "Extended warranty",
    "No foreign transaction fees"
  ]
}
```

```json
{
  "title": "Which Chase Freedom Credit Card Should You Get?",
  "author": "NerdWallet",
  "date": "2023-10",
  "summary": "A guide to choosing the best Chase Freedom credit card based on individual spending habits and rewards preferences.",
  "key_points": [
    "Overview of Chase Freedom credit card options.",
    "Comparison of benefits and rewards among different Chase Freedom cards.",
    "Tips for maximizing rewards based on spending categories.",
    "Considerations for new applicants and existing Chase customers."
  ]
}
```